<a href="https://colab.research.google.com/github/REICHIYAN/REI/blob/master/AlgoTrade.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
#!pip install git+https://github.com/pydata/pandas-datareader.git

In [0]:
from pandas_datareader import data

start_date = '2014-01-01'
end_date = '2018-01-01'

goog_data = data.DataReader('GOOG', 'yahoo', start_date, )

In [41]:
print(goog_data)

                   High          Low  ...     Volume    Adj Close
Date                                  ...                        
2014-01-02   556.788025   552.060730  ...  3656400.0   554.481689
2014-01-03   556.379578   550.401978  ...  3345800.0   550.436829
2014-01-06   557.340942   551.154114  ...  3551800.0   556.573853
2014-01-07   567.717041   558.486633  ...  5124300.0   567.303589
2014-01-08   571.517822   564.528992  ...  4501700.0   568.484192
...                 ...          ...  ...        ...          ...
2020-05-15  1374.479980  1339.000000  ...  1707700.0  1373.189941
2020-05-18  1392.324951  1354.250000  ...  1824000.0  1383.939941
2020-05-19  1392.000000  1373.484985  ...  1280600.0  1373.484985
2020-05-20  1410.420044  1387.250000  ...  1654200.0  1406.719971
2020-05-21  1415.489990  1397.760010  ...   332513.0  1400.729980

[1608 rows x 6 columns]
